# Part 4: Data Wrangling - Using Dask to remove repeating values

Currently, we have parquet files for each associted json files that matches a discount rate for a procedure(code type + code value) to a CCN (hospital). To make this data more managable, we need to remove data that is repeated.

This notebook utilizes dask to delete repeated values to make a more manageable data set. After this we can select for some procedure and begin exploratory analysis. We begin by importing packages we intend to use.

In [2]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
from datetime import date, datetime, timedelta
import sqlalchemy
import pymysql
import openpyxl
import glob
from ast import literal_eval
from collections import Counter
from tqdm.auto import tqdm
import dask.dataframe as dd
import dask.array as da
import dask.bag as db
import pyarrow as pa
from dask.distributed import Client

Our dotenv contains our location of where our previous files from prior notebooks are stored. We will access these below.

In [3]:
load_dotenv()

hyperlink_path = 'json_completed_hyperlinks_update.csv'
parent_dir = os.getenv('dir')
data_dir = os.path.join(parent_dir,'data_update')

df = pd.read_csv(hyperlink_path, header=None)
df.head()
df.columns = ['ParseID','Hyperlink']
hyperlinks = df['Hyperlink'].tolist()

def foldername(hyperlink):
    hyperlink = hyperlink.split('/')[-1]
    return hyperlink[0:-8]
def providers_path(folder):
    return os.path.join(data_dir,folder,folder+'_providers.csv')

folder_names= [foldername(hyperlink) for hyperlink in hyperlinks]
provider_files = [providers_path(folder_name) for folder_name in folder_names]

Using dask allows us to work in parallel when completing tasks. We can select the number of workers. I set it to one here due to the CPU I am working with.

In [4]:
client = Client(n_workers=1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 4,Total memory: 3.86 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:53927,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 3.86 GiB
Comm: tcp://127.0.0.1:53937,Total threads: 4
Dashboard: http://127.0.0.1:53938/status,Memory: 3.86 GiB
Nanny: tcp://127.0.0.1:53930,


Dask has similar functions to panda but does not perform the task until it is required to. This is a form of lazy computation. When I call this ddf, it displays basic information about the tables I want to gather.

In [5]:
ddf = dd.read_parquet('D://Vignesh/Capstone/data_update/*/*_merge.parquet', 
                     columns=['billing_type','billing_code','negotiated_rates','ccn'], engine='pyarrow')

ddf


,billing_type,billing_code,negotiated_rates,ccn
npartitions=586,,,,
,object,object,float64,object
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


Now we will drop the duplicates from our file. This will save the file as a CSV for acess later. You may save the file as a different format.

In [7]:
%%time

ddf = ddf.drop_duplicates(ignore_index=True)
ddf.to_csv("D://Vignesh/Capstone/combined/export.csv")

MemoryError: Unable to allocate 179. MiB for an array with shape (23446002,) and data type int64

2023-02-18 12:29:20,183 - distributed.nanny - WARNING - Restarting worker


### SQL database (Optional)
The following code can be used to set up a database with these values and is optional. This can be useful when exploring different procedures.
Lets determine varchars lengths for each column we intend to create.

In [11]:
n = ddf['billing_type'].nunique().compute()
n

In [8]:
unquie_types = ddf['billing_type'].unique().compute()
bt_max = ddf['billing_type'].str.len().max().compute()
bc_max = ddf['billing_code'].str.len().max().compute()
ccn_max = ddf['ccn'].str.len().max().compute()

In [9]:
print('The unique types are :{0}'.format(unquie_types))

The max length for ccn is :6


In [6]:
(host, user, password, port, database) = (os.getenv('host'), os.getenv('user'), os.getenv('passwd'), os.getenv('port'), os.getenv('database'))

def get_connection():
    return sqlalchemy.create_engine(
        url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(
            user, password, host, port, database
        )
    )

engine = get_connection()

In [8]:
table= 'rates'
columns = ['billing_type', 'billing_code', 'negotiated_rates', 'ccn']
sqltypes_rates = {'billing_type': sqlalchemy.types.VARCHAR(length=8), 'billing_code': sqlalchemy.types.VARCHAR(length=7), 
                  'negotiated_rates': sqlalchemy.types.FLOAT(), 'ccn': sqlalchemy.types.VARCHAR(length=6)}

In [ ]:
ddf.head()

In [13]:
ddf.to_sql(name=table,uri="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(user, password, host, port, database), if_exists='append', index=False, chunksize=100, dtype=sqltypes_rates)